# Evaluate regression model

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecay import database
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import run
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', 
                              limit_score=0.9, exclude_table_name='rolldecay_exclude')

df_rolldecay_linear = database.load(rolldecay_table_name='rolldecay_linear_b', 
                              limit_score=0.9, exclude_table_name='rolldecay_exclude')

df_regression = database.load(rolldecay_table_name='rolldecay_quadratic_b_regression', 
                              limit_score=-np.inf, exclude_table_name='rolldecay_exclude')

df_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', 
                              limit_score=-np.inf, exclude_table_name='rolldecay_exclude')

df_rolldecay['g']=9.81
df_rolldecay['rho']=1000
df_rolldecay['Disp']=df_rolldecay['Volume']/(df_rolldecay['scale_factor']**3)
df_rolldecay['beam']=df_rolldecay['beam']/(df_rolldecay['scale_factor'])
df_rolldecay['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_rolldecay)
df_rolldecay['B_2_hat'] = run(function=lambdas.B_2_hat_lambda, inputs=df_rolldecay)

df_rolldecay_linear['g']=9.81
df_rolldecay_linear['rho']=1000
df_rolldecay_linear['Disp']=df_rolldecay_linear['Volume']/(df_rolldecay_linear['scale_factor']**3)
df_rolldecay_linear['beam']=df_rolldecay_linear['beam']/(df_rolldecay_linear['scale_factor'])
df_rolldecay_linear['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_rolldecay_linear)
df_rolldecay_linear['B_2_hat'] = 0

df_regression['g']=9.81
df_regression['rho']=1000
df_regression['Disp']=df_regression['Volume']/(df_regression['scale_factor']**3)
df_regression['beam']=df_regression['beam']/(df_regression['scale_factor'])
df_regression['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_regression)
df_regression['B_2_hat'] = run(function=lambdas.B_2_hat_lambda, inputs=df_regression)

df_ikeda['g']=9.81
df_ikeda['rho']=1000
df_ikeda['Disp']=df_ikeda['Volume']/(df_ikeda['scale_factor']**3)
df_ikeda['beam']=df_ikeda['beam']/(df_ikeda['scale_factor'])
df_ikeda['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_ikeda)
df_ikeda['B_2_hat'] = run(function=lambdas.B_2_hat_lambda, inputs=df_ikeda)



In [ ]:
#df_rolldecay['phi_a']=df_rolldecay['phi_start'].abs()
phi_a=3.4
df_rolldecay['phi_a']=np.deg2rad(phi_a)
df_rolldecay['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_rolldecay)

df_rolldecay_linear['phi_a']=df_rolldecay_linear['phi_start'].abs()
df_rolldecay_linear['B_2'] = 0
df_rolldecay_linear['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_rolldecay_linear)

#df_regression['phi_a']=df_regression['phi_start'].abs()
df_regression['phi_a']=np.deg2rad(phi_a)
df_regression['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_regression)

In [ ]:
df_rolldecay[['B_e','B_1','B_2','beam','Disp','phi_a']].head()

In [ ]:
df_rolldecay_linear[['B_e','B_1','B_2']].head()

In [ ]:
df = pd.merge(left=df_rolldecay, right=df_rolldecay_linear,how='inner',
              left_index=True, right_index=True, suffixes=('','_linear'))

In [ ]:
fig,ax=plt.subplots()
df.plot(x='B_1_linear', y=['B_e','B_1'], style='.', ax=ax)
ax.grid()

In [ ]:
mask=df['B_1']<0
df.loc[mask]

In [ ]:
df_regression[['score']].head()

In [ ]:
df_regression[['B_e','B_1','B_2']].head()

In [ ]:
df_regression.describe()

In [ ]:
df_compare = pd.merge(left=df_rolldecay, right=df_regression, how='inner', left_index=True, right_index=True,
                     suffixes=('','_regression'))

df_compare = pd.merge(left=df_compare, right=df_ikeda, how='inner', left_index=True, right_index=True,
                     suffixes=('','_ikeda'))



In [ ]:
mean_squared_error(y_true=df_compare['B_e'], y_pred=df_compare['B_e_regression'])

In [ ]:
r2_score(y_true=df_compare['B_e'], y_pred=df_compare['B_e_regression'])

In [ ]:
df_compare['score_regression'].mean() - 2*df_compare['score_regression'].std()

In [ ]:
phi_as = np.deg2rad(np.linspace(1,8,30))
scores = []

for phi_a in phi_as:

    df_rolldecay_ = df_rolldecay.copy()
    df_regression_ = df_regression.copy()
    
    df_rolldecay_['phi_a']=phi_a
    df_rolldecay_['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_rolldecay_)
    
    df_regression_['phi_a']=phi_a
    df_regression_['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_regression_)
    
    df_compare_ = pd.merge(left=df_rolldecay_, right=df_regression_, how='inner', left_index=True, right_index=True,
                     suffixes=('','_regression'))
    
    score = r2_score(y_true=df_compare_['B_e'], y_pred=df_compare_['B_e_regression'])
    scores.append(score)

In [ ]:
fig,ax=plt.subplots()
ax.plot(np.rad2deg(phi_as),scores)    

In [ ]:
8/(3*np.pi)

In [ ]:
np.rad2deg(df_compare['phi_start'].abs()).mean()

In [ ]:
interesting = ['score','score_regression','score_ikeda']
df_compare[interesting].describe()

In [ ]:
interesting = ['B_1_hat','B_1_hat_regression','B_1_hat_ikeda']
df_compare[interesting].describe()

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_e',y=['B_e_regression','B_e_ikeda'], ax=ax, style='o', alpha=0.3)

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_1_hat',y=['B_1_hat_regression','B_1_hat_ikeda'], ax=ax, style='o', alpha=0.3)

In [ ]:
interesting = ['score','score_regression','score_ikeda']
mask = df_compare['TA']/df_compare['lpp'] > 0.034
df_compare.loc[mask][interesting].describe()

In [ ]:
fig,ax=plt.subplots()
df_compare[mask].plot(x='B_e',y=['B_e_regression','B_e_ikeda'], ax=ax, style='o', alpha=0.3)

In [ ]:
fig,ax=plt.subplots()
df_compare[mask].plot(x='B_1_hat',y=['B_1_hat_regression','B_1_hat_ikeda'], ax=ax, style='o', alpha=0.3)

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_2_hat',y=['B_2_hat_regression','B_2_hat_ikeda'], ax=ax, style='o', alpha=0.3)

In [ ]:
df_regression.describe()

In [ ]:
df_ikeda.describe()

In [ ]:
df_compare.describe()